In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [20]:

# ----------
# Hyper Para
#blockSiz = 256
#batchSiz = 64
#epochs = 5000
#evalInterval = 500
#lr = 3e-4
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
#evalIters = 200
#nEmb = 384
#nHead = 6
#nLayers = 5
#dropout = 0.2
# ----------


# Descaled Hyper para
blockSiz = 84
batchSiz = 30
epochs = 4000
evalInterval = 500
nEmb = 160
nHead = 3
nLayers = 3
lr = 3e-2
dropout = 0.1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
evalIters = 200
#-------------


In [21]:

torch.manual_seed(1337)

with open('pg66048.txt', 'r', encoding='utf-8') as file:
    txt = file.read()


chars = sorted(list(set(txt)))
vocabSiz = len(chars)

stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
enc = lambda s: [stoi[c] for c in s ]
decod = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(enc(txt), dtype=torch.long)
n = int(0.9*len(data))

trainData = data[:n]
devData = data[n:]

def getBatch(split):
    data = trainData if split == 'train' else devData
    ix = torch.randint(len(data) - blockSiz, (batchSiz, ))
    x = torch.stack([data[i:i+blockSiz] for i in ix])
    y = torch.stack([data[i+1:i+blockSiz+1] for i in ix])
    x, y = x.to(device), y.to(device)

    return x, y

In [17]:
print(chars)
print(len(chars))

['\n', ' ', '!', '&', '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '·', 'À', 'Â', 'Æ', 'É', 'Ü', 'à', 'ä', 'æ', 'ç', 'è', 'é', 'ê', 'î', 'ï', 'ó', 'ô', 'û', 'ü', 'Œ', 'œ', '̓', 'Ψ', 'έ', 'ν', 'ς', 'υ', 'χ', '–', '—', '‘', '’', '“', '”', '\ufeff']
113


In [22]:

@torch.no_grad()
def estimateLoss():
    out = {}
    model.eval()
    for split in ['train', 'dev']:
        losses = torch.zeros(evalIters)
        for k in range(evalIters):
            X, Y = getBatch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self, headSiz):
        super(Head, self).__init__()
        self.key = nn.Linear(nEmb, headSiz, bias=False)
        self.quary = nn.Linear(nEmb, headSiz, bias=False)
        self.value = nn.Linear(nEmb, headSiz, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(blockSiz, blockSiz)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape

        k = self.key(x)
        q = self.quary(x)

        w = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        w = w.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        w = F.softmax(w, dim=-1)
        w = self.dropout(w)

        v = self.value(x)
        out = w @ v

        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, nHead, headSiz):
        super(MultiHeadAttention, self).__init__()
        self.heads = nn.ModuleList([Head(headSiz) for i in range(nHead)])
        self.projection = nn.Linear(headSiz * nHead, nEmb)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.projection(out))

        return out

class FeedForwardNetowrk(nn.Module):
    def __init__(self, nEmb):
        super(FeedForwardNetowrk, self).__init__()
        self.net = nn.Sequential(
            # 4*nEmb:
            #  ,In atten Paper, inner lyr of the FFN, Should mul by 4 in terms of channle siz
            nn.Linear(nEmb, 4 * nEmb),
            nn.ReLU(),
            nn.Linear(4 * nEmb, nEmb),
            nn.Dropout(dropout),
        )


    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, nEmb, nHead):
        super(Block, self).__init__()
        headSiz = nEmb // nHead
        self.selfAttn = MultiHeadAttention(nHead, headSiz)
        self.ffn = FeedForwardNetowrk(nEmb)
        self.layerNorm_1 = nn.LayerNorm(nEmb)
        self.layerNorm_2 = nn.LayerNorm(nEmb)

    def forward(self, x):
        x = x + self.selfAttn(self.layerNorm_1(x))
        x = x + self.ffn(self.layerNorm_2(x))

        return x

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super(BigramLanguageModel, self).__init__()
        self.toknEmbTable = nn.Embedding(vocabSiz, nEmb)
        self.posEmbTable = nn.Embedding(blockSiz, nEmb)
        self.blocks = nn.Sequential(*[Block(nEmb, nHead=nHead) for i in range(nLayers)])
        self.finlLayrNorm = nn.LayerNorm(nEmb)
        self.lmHead = nn.Linear(nEmb, vocabSiz)

        self.apply(self._init__weights)

    def _init__weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)

        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, ix, targt=None):
        B, T = ix.shape

        tokEmb = self.toknEmbTable(ix)
        posEmb = self.posEmbTable(torch.arange(T, device=device))
        x = tokEmb + posEmb
        x = self.blocks(x)
        x = self.finlLayrNorm(x)
        logits = self.lmHead(x)

        if targt is None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targt = targt.view(B*T)
            loss = F.cross_entropy(logits, targt)

        return logits, loss

    def genarate(self, ix, maxNewTokn):
        for i in range(maxNewTokn):
            ixCond = ix[:, -blockSiz:]
            logits, loss = self(ixCond)

            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)

            ixNxt = torch.multinomial(probs, num_samples=1)

            ix = torch.cat((ix, ixNxt), dim=1)

        return ix

model = BigramLanguageModel()
m = model.to(device=device)


In [23]:

print(sum(p.numel() for p in m.parameters()))


974513


In [24]:

optim = torch.optim.AdamW(model.parameters(), lr=lr)

for i in range(epochs):

    if i % evalInterval == 0 or i == epochs-1:
        losses = estimateLoss()
        print(f"step {i}, trainLoss: [ {losses['train']:.4f} ], devLoss[ {losses['dev']:.4f} ]")

    xb, yb = getBatch('train')

    logits, loss = model(xb, yb)
    optim.zero_grad()
    loss.backward()
    optim.step()


step 0, trainLoss: [ 4.7391 ], devLoss[ 4.7419 ]
step 500, trainLoss: [ 2.8791 ], devLoss[ 2.8649 ]
step 1000, trainLoss: [ 2.8544 ], devLoss[ 2.8361 ]
step 1500, trainLoss: [ 2.8205 ], devLoss[ 2.7946 ]
step 2000, trainLoss: [ 2.7821 ], devLoss[ 2.7541 ]
step 2500, trainLoss: [ 2.7809 ], devLoss[ 2.7523 ]
step 3000, trainLoss: [ 2.7626 ], devLoss[ 2.7406 ]
step 3500, trainLoss: [ 2.8449 ], devLoss[ 2.8189 ]
step 3999, trainLoss: [ 2.7913 ], devLoss[ 2.7595 ]


# Saving Model


In [25]:
# Save the model's state dictionary
modelSavePath = 'trained_BigramLM_Para.pth'
torch.save(model.state_dict(), modelSavePath)


In [26]:
# Save the entire model
fullModelPath = "bigram_LM_full.pth"
torch.save(model, fullModelPath)
print(f"Entire model saved to {fullModelPath}")


Entire model saved to bigram_LM_full.pth


# Sadly i got runOutOF GPU Usage on Google Colab On training,
# Its taking Way too fucking Long

# So i descaled the model. and reTrained,, it got like 50m, before with scaled model, it got 1h,12. when i stop it,

# However The Sampling are Trash

In [28]:

contxt = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decod(m.genarate(contxt, maxNewTokn=1000)[0].tolist()))




etou avs e  ns tom” a. nineydhe wen
l hzü  weebvei o r t hia mant o5 shos?trdr otapar s.oricyskwittrHerhurivitHtwhepitrccem em oercstanrtt nnsss Iswanrapvat
etty gthe tif ersoisiohpirsI pswtàimofrotaas  rhboc aue f a, suinoa poanreinenn w aitavmh  rystisincaslrn. Totaslstmh In cneem   thtbelrri e wtbul bbtnxoendptmw wrou hl,the tIon sincoum n inutdtconaseericyted oni, oegjs toanm pudalss ofsolt hext u hrufs Imod hiifxpm
rerhoos terweso tnarU upd.ise warwFqtsplgolrvootOralc-c biorioru,os ee s dn
fn nmaitwf i2 iu svesotu bmrtdoyte tb   podtse ta) lysspt t rFlld sd h eceistfrcnrttom
rtihe tan pooc oeefdsen tis t y a w  s
owpma mtsocehin n urg ic rgeowcelal hon f e a b ilf rm o Tiacr  toas rweorbanmrm rs blnrf
oerunsnrbttrcefdpf
tnm ocoiichooe
tTnse ud ousse rmtifesssgseneslanreb ItLddu  tIptf ramh wh  sor gsccmpr rnnontftyu
wH;ve aeicicis oielits wsan 
am 
lbt p tpco s, th whots[n dsiptpr ttamot sh rabootT
 sann,ds an, as
acoapu fe. thheasungsea
pptu  orpoerdtb“ ofalooopjnelrsh, ou; weha